<img src="AUEB.png" /> <img src="MSc_BA.png" />

# Athens University of Economics and Business
# School of Business
# Department of Management Science & Technology
# Master of Science in Business Analytics
___

<table style='float:left;font-size: 20px;'>
    <tr>
        <th style='text-align: left;'>Program:</th>
        <td style='text-align: left;'>Full-Time</td>
    </tr>
    <tr>
        <th style='text-align: left;'>Quarter:</th>
        <td style='text-align: left;'>3rd (Spring Quarter)</td>
    </tr>
    <tr>
        <th style='text-align: left;'>Course:</th>
        <td style='text-align: left;'>Mining Big Datasets</td>
    </tr>
    <tr>
        <th style='text-align: left;'>Assignment:</th>
        <td style='text-align: left;'>Assignment 1</td>
    </tr> 
    <tr>
        <th style='text-align: left;'>Students (Registration No):</th>
        <td style='text-align: left;'>f2822202, f2822217</td>
    </tr>
</table>

# Assignment 1
## 1) Import and pre-process the dataset with users

Firstly, we imported all necessary libraries:

- `pandas` (imported as `pd`) for data manipulation and analysis.
- `numpy` (imported as `np`) for numerical operations.
- `random` for generating random numbers.
- `IPython.display` for displaying data in a Jupyter Notebook or IPython environment.

Then, we read and displayed data:

- We read the contents of the "users.txt" file and assigned it to a variable named `users`. The file has pipe-separated values (|) and no header row. The columns are labeled as "id", "age", "gender", "occupation", and "postcode". Then, we displayed the `users` DataFrame. It contains 943 users with their IDs and all other included information.

- We read the contents of the "movies.txt" file and assigned it to a variable named `movies`. The file has pipe-separated values (|) and no header row. Only the first two columns (index 0 and 1) are read and labeled as "id" and "movie". Then, we displayed the `movies` DataFrame. It contains 1,682 movies with their IDs and title.

- We read the contents of the "ratings.txt" file and assigned it to a variable named `ratings`. The file has tab-separated values (\t) and no header row. Only the first two columns (index 0 and 1) are read and labeled as "userid" and "movieid". Then, we displayed the `ratings` DataFrame. It contains 100,000 pairs of a single movie that a single user rated.

**Note**: The files "users.txt", "movies.txt", and "ratings.txt", along with the "AUEB.png" and the "MSc_BA.png" (used in the first cell) must exist in the same directory as the Jupyter notebook.

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import random
from IPython.display import display


# Read and display data:
# users
users = pd.read_csv("users.txt", sep="|", header=None, names=["id", "age", "gender", "occupation", "postcode"])
display(users)

# movies
movies = pd.read_csv('movies.txt', sep='|', encoding='latin-1', usecols=[0,1], names=["id","movie"])
display(movies)

# ratings
ratings = pd.read_csv("ratings.txt", sep="\t", usecols=[0,1], names=["userid", "movieid"])
ratings
print("---------------------------------------------------------------------")

,id,age,gender,occupation,postcode
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213
...,...,...,...,...,...
938,939,26,F,student,33319
939,940,32,M,administrator,02215
940,941,20,M,student,97229
941,942,48,F,librarian,78209


,id,movie
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)
...,...,...
1677,1678,Mat' i syn (1997)
1678,1679,B. Monkey (1998)
1679,1680,Sliding Doors (1998)
1680,1681,You So Crazy (1994)


---------------------------------------------------------------------


## 2) Compute exact Jaccard similarity of users

Firstly, we defined a class called Pair with several methods. The `Pair` class represents a pair of users, where each user is identified by `user_i` and `user_j`. The class has the following attributes:

- `user_i`: Represents the identifier of the first user in the pair.
- `user_j`: Represents the identifier of the second user in the pair.
- `exact`: Represents the exact Jaccard similarity associated with the pair of users. It is initially set to 0.

The class has the following methods:

- `__init__(self, i, j)`: This is the constructor method of the `Pair` class. It is called when a new instance of the class is created. It takes two arguments, i and j, which are used to initialize the `user_i`, `user_j` and `exact` attributes of the class.
- `set_exact(self, jac)`: This method sets the exact attribute of the `Pair` object to the exact Jaccard similarity, specified by the `jac` parameter.

In [2]:
class Pair:
    def __init__(self, i, j):
        self.user_i = i
        self.user_j = j
        self.exact = 0
#         self.approx=0
        
    def set_exact(self, jac):
        self.exact = jac
        
#     def set_approx(self,aprox_jac):
#         self.approx = aprox_jac

Then, we performed the following actions:

1. Initialized a dictionary named `dictofpairs` to store pairs of users and their associated `Pair` object.
2. Grouped the movie IDs that each user rated and stored the results in the `grouped` Series variable.
3. Iterated over the index and values of the `grouped` variable to:
    - Calculate the intersection and union of the movie IDs for each pair of users.
    - Calculate their Jaccard similarity by dividing the intersection by the union.
    - Create a `Pair` object using the `Pair` class and set its `exact` attribute to the calculated Jaccard similarity.
    - Add the `Pair` object to the `dictofpairs` dictionary, using the pair of user IDs as the key and the `Pair` object as the value.
    - Denote the pair of users that their calculated Jaccard similarity is greater than or equal to 50% and append it to the `list_exact` list.
    - Find the most similar pair of users (the one with the highest Jaccard similarity), and
   
   
4. Output the movie titles that the most similar pair rated.

In [3]:
# Initialize pair of users dictionary
dictofpairs = {}

# Group movie IDs by user IDs
grouped = ratings.groupby('userid')['movieid'].apply(set)

list_exact = []
maxJac = 0
k = 0
print("The pairs of users that have similarity at least 50% are:")
for i, user_i in enumerate(grouped.index[:-1]): 
    for user_j in grouped.index[i+1:]:
        
        # Calculate Jaccard similarity for all pairs of users
        intersection = len(grouped[user_i] & grouped[user_j])
        union = len(grouped[user_i] | grouped[user_j])
        jaccard = intersection / union
        
        #Create pair object
        pair = Pair(user_i,user_j)
        pair.set_exact(jaccard)
        dictofpairs[(user_i, user_j)] = pair # add the object to the dictionary
        if jaccard >= 0.5:
            k+=1
            print(k, ".\t(", user_i, "-", user_j, ") has Jaccard similarity: ", jaccard, sep="")
            list_exact.append((user_i, user_j)) # list contains the exact similarities >= 0.5
            if maxJac < jaccard:
                maxJac = jaccard
                maxPair = pair
                # max_set contains union of movies of max simiilarity pair
                max_set = grouped[user_i] & grouped[user_j]
print("---------------------------------------------------------------------")
print("The most similar pair is (", maxPair.user_i, "-", maxPair.user_j, ") and their common movie titles are:", sep="")
print(movies.movie.loc[movies.id.isin(max_set)].to_string(index=False))
print("---------------------------------------------------------------------")

The pairs of users that have similarity at least 50% are:
1.	(197-600) has Jaccard similarity: 0.5
2.	(197-826) has Jaccard similarity: 0.512987012987013
3.	(328-788) has Jaccard similarity: 0.6729559748427673
4.	(408-898) has Jaccard similarity: 0.8387096774193549
5.	(451-489) has Jaccard similarity: 0.5333333333333333
6.	(489-587) has Jaccard similarity: 0.6299212598425197
7.	(554-764) has Jaccard similarity: 0.5170068027210885
8.	(600-826) has Jaccard similarity: 0.5454545454545454
9.	(674-879) has Jaccard similarity: 0.5217391304347826
10.	(800-879) has Jaccard similarity: 0.5
---------------------------------------------------------------------
The most similar pair is (408-898) and their common movie titles are:
                                  Kolya (1996)
                                Contact (1997)
                                Gattaca (1997)
                      Starship Troopers (1997)
                      Good Will Hunting (1997)
                   English Patient, T

## 3) Compute similarity using Min-hash signatures

Firstly, we defined two classes, `User` and `Signatures`, along with their methods. Below's an explanation of what each class and method does:

1. `User` class:
   - The `__init__` method initializes a `User` object with a `user` identifier and a `num` parameter. It also creates a NumPy array called `user_signature` with `num` elements (the number of hash functions), initialized to zero.

2. `Signatures` class:
   - The `__init__` method initializes a `Signatures` object with a `num` parameter (the number of hash functions) and an `R` parameter (R a large enough prime number). It also initializes other attributes such as `a`, `b`, and `R`, for the hash function, `combination` as a set to make sure that each hash function uses different values of a,b pairs, and `list_approx` list for storing pairs that have estimated similarity at least 50%.
   - The `convert` method is a hash function that maps each element `x` to a value based on the hash parameters `a`, `b`, and `R`. It returns the minimum value obtained from the hash calculation i.e., the min-hash value.
   - The `create_sig` method creates user signatures for a given `grouped` object. It initializes a list of `User` objects called `listofusers`, where each `User` object is assigned a unique identifier. It iterates `num` times and generates random hash parameters `a` and `b`. It calculates the hash values for each user in `grouped` using the `convert` method and assigns the minimum hash value to the corresponding `User` object's `user_signature` array.
   - The `assess_similarity` method compares the user signatures of each pair of users in `listofusers`. If the similarity, defined as the ratio of matching elements in the user signatures, is greater than or equal to 0.5, the pair is added to the `list_approx`.
   - The `LSH_similarity_evaluations` method evaluates the similarity between users using Locality Sensitive Hashing (LSH). It reshapes the user signatures 2-D array into a new 3-D array by splitting an array of (users * total number of hash functions) into (users * `bands` groups * `r` hash functions) and iterates over the pairs of users. It compares the rows of the reshaped array and if there is a match, it adds the pair to the `list_approx` and adds one to the variable `summ` responsible for counting the candidate pairs.
   - The `calculate_false` method calculates false positives and false negatives by comparing the `list_approx` with the actual positive Pairs stored in the `list_exact` variable.
   - The `calculate_true` method calculates true positives by comparing the `list_approx` with the forementioned `list_exact` variable.

**Note**: The `LSH_similarity_evaluations` and `calculate_true`methods will be used for answering the fourth task.

In [4]:
class User:
    
    def __init__(self, user, num):
        self.user = user
        self.user_signature = np.zeros(num)    

        
class Signatures:
    
    def __init__(self, num, R):
        self.num = num
        self.a = 0
        self.b = 0
        self.R = R 
        self.combination = set()
        self.list_approx=[]
        
    #this function is used to map each element of the series to the hash function
    def convert(self, x):
        return min((self.a * x + self.b) % self.R)

    def create_sig(self, grouped):
        self.listofusers = [User(i, self.num) for i in grouped.index] # initialise list
        i = 0
        while i < self.num:
            self.a = random.randint(1, self.R-1)
            self.b = random.randint(1, self.R-1)
            if (self.a, self.b) not in self.combination:
                self.combination.add((self.a, self.b))
                result = grouped.map(self.convert)
                for user_id, min_value in result.items():
                    self.listofusers[user_id-1].user_signature[i] = min_value # Here, we assume the users go from 1-943,
                                                                              # We could make it more dynamic if needed               
                i += 1
      
    def assess_similarity(self):  
        for i,user_i in enumerate(self.listofusers[:-1]): 
            for user_j in self.listofusers[i+1:]:
                similarity = np.count_nonzero(user_i.user_signature==user_j.user_signature)/self.num
                if similarity >= 0.5:
                    self.list_approx.append((user_i.user,user_j.user)) # this list contains pairs that has >= 0.5 approx_sim.

    def LSH_similarity_evaluations(self, bands):
        r = int(self.num/bands)
        x = np.zeros((len(self.listofusers), self.num)) 
        for i in range(len(self.listofusers)):
            x[i] = self.listofusers[i].user_signature        
        reshaped_arr = x.reshape(len(self.listofusers), bands, r)
        total_eval = 0
        summ = 0
        for i in range(len(self.listofusers)-1):
            for j in range(i+1,len(self.listofusers)):
                if np.count_nonzero(np.all(reshaped_arr[i] == reshaped_arr[j], axis=1)) > 0:
                    summ+=1
                    self.list_approx.append((self.listofusers[i].user, self.listofusers[j].user))
        return summ
    
    def calculate_false(self, list_exact):
        #Calculate false positive
        false_pos = 0
        for pair in self.list_approx:
            if pair not in list_exact:
                false_pos+=1  
        #Calculate false negative
        false_neg=0
        for pair in list_exact:
            if pair not in self.list_approx:
                false_neg+=1
        return (false_pos,false_neg)
    
    def calculate_true(self, list_exact):
        #Calculate true positive
        true_pos = 0
        for pair in self.list_approx:
            if pair in list_exact:
                true_pos+=1
        return true_pos

We then performed the following steps:

1. Grouped the 'movieid' column of the `ratings` dataframe by `userid`, converted the grouped data into arrays and assigned to the variable `grouped`.

2. Initialized a loop over a range of large enough prime values for the parameter `R`, greater than the number of movies, in order to finetune it.

3. Inside the loop, there is another loop over a range of values for the parameter `i`, which represents the number of hash functions. The loop iterates for `i = 50`, `i = 100`, and `i = 200`.

4. Within the nested loops, there is a loop that runs 5 times, represented by the variable `j`. This loop is used to calculate the average number of false positives and false negatives for each combination of `R` and `i`.

5. In each iteration of the inner loop, a `Signatures` object `c` is created with `i` hash functions and `R` value. Then, the `create_sig` method is called to create user signatures based on the `grouped` data.

6. The `assess_similarity` method is called to assess the similarity between users based on their min-hash signatures.

7. The `calculate_false` method is used to calculate the number of false positives and false negatives by comparing the calculated similarity results (`list_approx`) with the `list_exact` list calculated in the previous question.

8. The number of false positives and false negatives is accumulated over the 5 iterations of the inner loop.

9. After the inner loop completes, the average number of false positives and false negatives is printed for the given `R` and `i` values.

10. Finally, we checked if the sum of the average false positives and false negatives for the current `R` is smaller than the previously stored `sum_falses`. If it is, the current `R` value is assigned to `best_R`, and the `sum_falses` value is updated.

11. After the outer loop completes, the value of `best_R` is printed, which represents the `R` value that resulted in the smallest sum of false positives and false negatives. 

For random seed 1, we found the R with the lowest average (on 5 runs) sum of false positive and negative to be 4,003 and commented out the for loop iterating over the range of prime numbers in order for the code to have a faster run time. 

Finally, we reached to the conclusion that increasing the number of hash functions in min-hashing tends to reduce the false positive rate but does not directly impact false negatives. The choice of the number of hash functions should be based on the desired trade-off between accuracy, computational complexity, and the acceptable false positive rate.

False negatives in min-hashing depend on the similarity threshold used for identifying similar pairs of users. The number of hash functions in min-hashing affects the accuracy and false positive rate but not the false negatives. Choosing an appropriate similarity threshold is crucial for minimizing false negatives in min-hashing.

In [5]:
grouped = ratings.groupby('userid')['movieid'].apply(np.asarray)
#Calculate average of false positive and negative
random.seed(1)
best_R = 0
sum_falses = 1e+10
for R in [4003]: # smallest number of falses = 8.4 + 1.0
# for R in [4003, 8191, 131071, 6700417, 2147483647, 999999000001, 67280421310721]: # large prime numbers
    print("For R:", R)
    for i in [50,100,200]:
        false_pos=0
        false_neg=0
        for j in range(5):
            c = Signatures(i,R)
            c.create_sig(grouped)
            c.assess_similarity()
            pos,neg=c.calculate_false(list_exact)
            false_pos+=pos
            false_neg+=neg
        print("\nNumber of hash functions:", i)
        print("Average number of false positives:", false_pos/5)
        print("Average number of false negatives:", false_neg/5)
        if false_pos/5 + false_neg/5 < sum_falses:
            best_R = R
            sum_falses = false_pos/5 + false_neg/5
    print("----------------------------------------")
# print("\nThe best R is:", best_R)

For R: 4003

Number of hash functions: 50
Average number of false positives: 114.6
Average number of false negatives: 2.6

Number of hash functions: 100
Average number of false positives: 62.0
Average number of false negatives: 2.0

Number of hash functions: 200
Average number of false positives: 8.4
Average number of false negatives: 1.0
----------------------------------------


## 4) Locate similar users using LSH index

Firstly, we defined a function named `LSH` that performs Locality Sensitive Hashing (LSH) for approximate similarity evaluation between users. Here's a summary of what the code does:

1. It takes several parameters: `num` (number of hash functions), `b` (number of bands), `R` (large prime number), `grouped` (grouped per user ratings data), and `list_exact` (exact similarity pairs).
2. The code initializes variables for counting true positives and total similarity evaluations to zero.
3. The code enters a loop that iterates five times to perform LSH evaluation.
4. Within each iteration:
   * A new instance `c` of the `Signatures` class is created with `num` and `R`.
   * The method `create_sig` is called on `c` to create user signatures using the `grouped` ratings data.
   * The method `LSH_similarity_evaluations` is called on `c` with `b` number of bands to find and count the number of predicted positive pairs i.e., the similarity evaluations that will be performed using the initial representations.
   * The number of true positives is calculated using the `calculate_true` method of `c` with `list_exact`.
   * The number of true pairs returned (true positives) and the number of candidate pairs for similarity evaluation (predicted positives) are accumulated for averaging.
   
   
5. After the loop, the averages of true and predicted positives are printed.

In [6]:
def LSH(num, b, R, grouped, list_exact):
    random.seed(1)
    true_pos=0
    sim_eval=0
    for i in range(5):      
        c = Signatures(num,R)
        c.create_sig(grouped)
        pred_pos=c.LSH_similarity_evaluations(b)
        sim_eval+=pred_pos
        true=c.calculate_true(list_exact)
        true_pos+=true
    print("Average number of true positives:", true_pos/5)
    print("Average number of similarity evaluations:", sim_eval/5)

We then set the value of the variable `num` to 200, representing the number of hash functions to be used and with different values for bands and rows per band i.e., the specified LSH instances, we called the `LSH` function that was previously defined.

From the output, we can derive that for the same number of hash functions, using LSH with more bands of less hash functions each, increases the number of true pairs returned (true positives), thus having a greater probability of capturing all of the actual positive pairs. 

However, using the forementioned LSH instance i.e., Instance 2 instead of the former i.e., Instance 1, produces more candidate pairs (predicted positive) for assesing their true similarity using their initial representations, thus also increasing the pairs predicted falsely as similar (false positive).

In [7]:
num = 200
print("The number of hash functions is:", num)
print('--------------------------------')
print("| LSH Instance: 1 \t\t|")
b = 25
r = 8
print("| Bands:", b, "\t\t\t|")
print("| Hash functions per band:", r, "\t|")
print('--------------------------------')
print("")
LSH(num, b, best_R, grouped, list_exact)
print("")
print('--------------------------------')
print("| LSH Instance: 2 \t\t|")
b = 40
r = 5
print("| Bands:", b, "\t\t\t|")
print("| Hash functions per band:", r, "\t|")
print('--------------------------------')
print("")
LSH(num, b, best_R, grouped, list_exact)
print("-------------------------------------------------")

The number of hash functions is: 200
--------------------------------
| LSH Instance: 1 		|
| Bands: 25 			|
| Hash functions per band: 8 	|
--------------------------------

Average number of true positives: 3.8
Average number of similarity evaluations: 51.6

--------------------------------
| LSH Instance: 2 		|
| Bands: 40 			|
| Hash functions per band: 5 	|
--------------------------------

Average number of true positives: 9.0
Average number of similarity evaluations: 2244.6
-------------------------------------------------


Using LSH (Locality Sensitive Hashing), instead of directly comparing users on their true representations, offers a trade-off between accuracy and efficiency. Here are the gains and losses associated with using LSH:

Gains:
1. Efficiency: LSH allows for efficient similarity evaluation by reducing the number of pairwise comparisons required. It achieves this by using hash functions to group similar items together, allowing for faster identification of potential matches. As we reported above, even on the latter case that the number of similarity evaluations increased, it never reached the total number of similarity evaluations performed using the initial representations which is 444,153 (943*942/2).
2. Scalability: LSH is particularly beneficial in scenarios with large datasets and high-dimensional data. It significantly reduces the computational burden compared to direct pairwise comparisons, making it more scalable for large-scale applications.
3. Reduced Memory Usage: LSH typically requires storing only the compact hash signatures of the items rather than their full representations. This leads to lower memory requirements, which can be advantageous when dealing with memory-constrained systems.

Losses:
1. Approximation: LSH introduces an element of approximation. It may classify some dissimilar pairs as similar (false positives) or fail to identify some similar pairs (false negatives). The degree of approximation depends on the chosen parameters and the nature of the data. Therefore, LSH is not guaranteed to provide exact similarity results. As we reported above, even on the latter case that the number of true positive pairs increased, it did not reach the actual number of true pairs which is 10.
2. Quality of Similarity: The similarity measured through LSH is often a proximity measure rather than an exact representation. It relies on the choice of hash functions and the number of hash tables used, which can impact the quality of similarity assessments. This may lead to slightly different similarity rankings compared to the true representations.
3. Sensitivity to Parameters: LSH performance is influenced by the selection of parameters such as the number of hash functions and the number of bands and thus it involves a trade-off. As we reported previously, as we were increasing the bands, true positives were increasing, but the same happened to the false positives too (number of similarity evaluations minus the true positives). Thus, choosing inappropriate parameters can result in decreased accuracy or efficiency. 

In summary, LSH offers significant gains in terms of computational efficiency and scalability compared to direct comparisons. However, it introduces an approximation factor and may yield slightly different similarity results. The suitability of LSH depends on the specific application requirements, the acceptable trade-off between accuracy and efficiency, and the characteristics of the dataset.